In [1]:
import train as train
import generate
import eval
import train
from modules.config import read_config
import modules.plots as plots
from modules.datageneration import load_data, data_functions
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pyro.poutine as poutine

In [2]:
BASE_DIR = "./tests/sanity_check"
PROFILE = "sinus-(10,20)-ownguide"

#BASE_DIR = "./tests/x-dim-test"
#PROFILE = "test"

In [3]:
configs = read_config(f"{BASE_DIR}/config.ini")
dataset_configs = read_config(f"{BASE_DIR}/dataset_config.ini")

config = configs[PROFILE]
NAME = config["NAME"]
dataset_config = dataset_configs[config["DATASET"]]
DATASET_NAME = dataset_config["NAME"]

In [4]:
inference_model = train.make_inference_model(config, device="cpu")

odict_keys(['model.fc.0.0.linear.weight', 'model.fc.0.0.linear.bias', 'model.fc.1.linear.weight', 'model.fc.1.linear.bias', 'data', 'obs'])
dict_keys(['_pyro_name', '_pyro_context', '_pyro_params', '_pyro_samples', 'training', '_parameters', '_buffers', '_non_persistent_buffers_set', '_backward_hooks', '_is_full_backward_hook', '_forward_hooks', '_forward_pre_hooks', '_state_dict_hooks', '_load_state_dict_pre_hooks', '_load_state_dict_post_hooks', '_modules', 'sample_input', 'sites'])


In [7]:
inference_model.guide.model_fc_0_0_linear_bias_loc

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

In [ ]:
_, _, (x_test, y_test), (x_test_in_domain, y_test_in_domain), (x_test_out_domain, y_test_out_domain) = load_data(f"{BASE_DIR}/datasets/{DATASET_NAME}", load_train=False, load_val=False)

In [ ]:
with poutine.trace() as tr:
    inference_model.model(x_test[0])
for site in tr.trace.nodes.values():
    print(site["type"], site["name"], site["value"].shape)

sample model.fc.0.0.linear.weight torch.Size([32, 1])
sample model.fc.0.0.linear.bias torch.Size([32])
sample model.fc.1.linear.weight torch.Size([1, 32])
sample model.fc.1.linear.bias torch.Size([1])
sample data torch.Size([1])
sample obs torch.Size([1])


In [ ]:
inference_model.model

BayesianRegressor(
  (fc): PyroSequential(
    (0): PyroSequential(
      (0): BayesianLinear(
        (linear): PyroLinear(in_features=1, out_features=32, bias=True)
      )
      (1): ReLU()
    )
    (1): BayesianLinear(
      (linear): PyroLinear(in_features=32, out_features=1, bias=True)
    )
  )
)